# Nutrition Problem

Una nutricionista se encuentra en el proceso de decisión para establecer que cantidad de dos tipos de alimento (A y B) debe incorporar en una dieta sabiendo que el costo por libra de cada uno de ellos es $400 y $300, respectivamente.  Además, se ha establecido que una libra de alimento tipo A contiene 3 miligramos de vitaminas, 6 miligramos de minerales y 4 miligramos de proteínas;  mientras que una libra de alimento tipo B contiene 8 miligramos de vitaminas, 3 miligramos de minerales y 5 miligramos de proteínas. También se debe garantizar consumir como mínimo 240 mg de vitaminas, 120 mg de minerales y 200 mg de proteínas. 
